>get access to google drive to store files there

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


> Impots all the requirments in other cells



In [2]:
import pandas as pd
! pip install pickle5
import pickle5 as pickle
import itertools
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical
import numpy as np
from re import sub, findall

     |████████████████████████████████| 133kB 13.1MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp36-cp36m-linux_x86_64.whl size=218620 sha256=f010c3735f9b0e2338915d42144ab337e4088324a22c68632a3b34acc378d9c1
  Stored in directory: /root/.cache/pip/wheels/a6/90/95/f889ca4aa8b0e0c7f21c8470b6f5d6032f0390a3a141a9a3bd
Successfully built pickle5


>constants values

In [3]:
diacritics = '‌|‮|‫|‬|⁩|⁯|⁭|­|‎|‍|‏|‪|⁠|⁧|्ۭۤिۗੱٗਿٰٞۚٓ°°·▫¸：°:⋅`▪̋ˆ¸ˈʿًٌٍَُِّءۙ˝ْٖۢٔۖۡ⋅'
punctuations = '[!"#$%&\'()٭*+-/;<=>@[\\]_{|}~£©«®±»†…•←−≠▌▐▬◊●◾♥♦✅✔➡《》口 员⁧⁩子宇平日本椅火‌窗箭航面곤쌍절￼�🔴‌🔶🔸🔹🖋🥇🥈🥉⁦‹›⁄É×àáãäçèéê→­ë„ì﴿™⌘▪`◄被♫ïñšн^○æ¼ř´ˈ²ā‐стćЭӨ§л✓˝ирБ½↓Чбуоɔøɣагəâдэ者ēÄγМ α爆﴾❤⁣ī″Á️óöсˉйí¤↑ɪ÷ûüý\\\ĀčěğİıłŞŠḤṣ–—Ẓū،‘’۞“”٪٫٬]'

zero_space = u'\u200C|\u200B|\uFEFF|\u180E'

given_doc_root_path = 'drive/MyDrive/Assignment_03/data/News/'
document_root_path = 'drive/MyDrive/Assignment_03/documents/'

>language toolkits

In [ ]:
from re import sub, findall

def normalizer(news):
    # remove english letters, special characters and punctuations
    news = sub('[A-Za-z]', ' ', news)
    # map all the used character to an uniform one given in simple_mapping in order to homogenize words
    news = sub_alphabets(news)
    for ch in punctuations:
        news = news.replace(ch, ' ')
    for ch in diacritics:
        news = news.replace(ch, '')

    # replace all the numbers with 'N'
    dg = findall(r"\d+", news)
    dg = sorted(dg, key=len, reverse=True)

    for num in dg:
        news = news.replace(str(num), ' N ')
    news = news.replace('.', '. ').replace('?', ' ؟').replace('؟', ' ؟').replace('؛', ' ؛')
    return ' '.join(news.split())


# #     based on parsivar's Normalizer().sub_alphabets() function and changed it in the needed way
def sub_alphabets(doc_string):
    # try:
    #     doc_string = doc_string.decode('utf-8')
    # except UnicodeEncodeError:
    #     pass

    #  #    edited parsivar's function based on whats interested in this assignment
    #  #    example: turn 'آسمان' into 'اسمان' as they have similar meaning and we do not consider only the correct written form
    a0 = 'ء|ﺌ|ﺋ'
    b0 = 'ئ'
    c0 = sub(a0, b0, doc_string)
    a1 = r'ﺎ|ﺄ|ٲ|ٱ|إ|ﺍ|أ'
    a11 = r'ﺁ|آ'
    b1 = r'ا'
    b11 = r'آ'
    c11 = sub(a11, b1, c0)
    c1 = sub(a1, b1, c11)
    a2 = r'ﺐ|ﺏ|ﺑ'
    b2 = r'ب'
    c2 = sub(a2, b2, c1)
    a3 = r'ﭖ|ﭗ|ﭙ|ﺒ|ﭘ'
    b3 = r'پ'
    c3 = sub(a3, b3, c2)
    a4 = r'ﭡ|ٺ|ٹ|ﭞ|ٿ|ټ|ﺕ|ﺗ|ﺖ|ﺘ|ﺜ'
    b4 = r'ت'
    c4 = sub(a4, b4, c3)
    a5 = r'ﺙ|ﺛ'
    b5 = r'ث'
    c5 = sub(a5, b5, c4)
    a6 = r'ﺞ|ﺝ|ڃ|ﺠ|ﺟ'
    b6 = r'ج'
    c6 = sub(a6, b6, c5)
    a7 = r'ڃ|ﭽ|ﭼ|ﭻ'
    b7 = r'چ'
    c7 = sub(a7, b7, c6)
    a8 = r'ﺢ|ﺤ|څ|ځ|ﺣ'
    b8 = r'ح'
    c8 = sub(a8, b8, c7)
    a9 = r'ﺥ|ﺦ|ﺨ|ﺧ'
    b9 = r'خ'
    c9 = sub(a9, b9, c8)
    a10 = r'ڏ|ډ|ﺪ|ﺩ'
    b10 = r'د'
    c10 = sub(a10, b10, c9)
    a11 = r'ﺫ|ﺬ|ﻧ'
    b11 = r'ذ'
    c11 = sub(a11, b11, c10)
    a12 = r'ۯ|ڙ|ڗ|ڒ|ڑ|ڕ|ﺭ|ﺮ'
    b12 = r"ر"
    c12 = sub(a12, b12, c11)
    a13 = r"ﺰ|ﺯ"
    b13 = r"ز"
    c13 = sub(a13, b13, c12)
    a14 = r"ﮊ|ﮋ"
    b14 = r"ژ"
    c14 = sub(a14, b14, c13)
    a15 = r"ݭ|ݜ|ﺱ|ﺲ|ښ|ﺴ|ﺳ"
    b15 = r"س"
    c15 = sub(a15, b15, c14)
    a16 = r"ﺵ|ﺶ|ﺸ|ﺷ"
    b16 = r"ش"
    c16 = sub(a16, b16, c15)
    a17 = r"ﺺ|ﺼ|ﺻ"
    b17 = r"ص"
    c17 = sub(a17, b17, c16)
    a18 = r"ﺽ|ﺾ|ﺿ|ﻀ"
    b18 = r"ض"
    c18 = sub(a18, b18, c17)
    a19 = r"ﻁ|ﻂ|ﻃ|ﻄ"
    b19 = r"ط"
    c19 = sub(a19, b19, c18)
    a20 = r"ﻆ|ﻇ|ﻈ"
    b20 = r"ظ"
    c20 = sub(a20, b20, c19)
    a21 = r"ڠ|ﻉ|ﻊ|ﻌ|ﻋ"
    b21 = r"ع"
    c21 = sub(a21, b21, c20)
    a22 = r"ﻎ|ۼ|ﻍ|ﻐ|ﻏ"
    b22 = r"غ"
    c22 = sub(a22, b22, c21)
    a23 = r"ﻒ|ﻑ|ﻔ|ﻓ"
    b23 = r"ف"
    c23 = sub(a23, b23, c22)
    a24 = r"ﻘ|ﻕ|ڤ|ﻖ|ﻗ"
    b24 = r"ق"
    c24 = sub(a24, b24, c23)
    a25 = r"ڭ|ﻚ|ﮎ|ﻜ|ﮏ|ګ|ﻛ|ﮑ|ﮐ|ڪ|ك|ﻙ"
    b25 = r"ک"
    c25 = sub(a25, b25, c24)
    a26 = r"ﮚ|ﮒ|ﮓ|ﮕ|ﮔ"
    b26 = r"گ"
    c26 = sub(a26, b26, c25)
    a27 = r"ﻟ|ﻝ|ﻞ|ﻠ|ڵ"
    b27 = r"ل"
    c27 = sub(a27, b27, c26)
    a28 = r"ﻡ|ﻤ|ﻢ|ﻣ"
    b28 = r"م"
    c28 = sub(a28, b28, c27)
    a29 = r"ڼ|ﻦ|ﻥ|ﻨ"
    b29 = r"ن"
    c29 = sub(a29, b29, c28)
    a30 = r"ۆ|ވ|ﯙ|ۈ|ۋ|ﺆ|ۊ|ۇ|ۏ|ۅ|ۉ|ﻭ|ﻮ|ؤ|ٶ|ﺅ"
    b30 = r"و"
    c30 = sub(a30, b30, c29)
    a31 = r"ﺔ|ﻬ|ھ|ﻩ|ﻫ|ﻪ|ۀ|ە|ة|ہ|ۃ"
    b31 = r"ه"
    c31 = sub(a31, b31, c30)
    a32 = r"ئ|ﭛ|ﻯ|ۍ|ﻰ|ﻱ|ﻲ|ں|ﻳ|ﻴ|ﯼ|ې|ﯽ|ﯾ|ﯿ|ێ|ے|ى|ي"
    b32 = r"ی"
    c32 = sub(a32, b32, c31)
    a33 = r'¬'
    b33 = r'‌'
    c33 = sub(a33, b33, c32)
    # pa0 = r'•|·|●|·|・|∙|｡|ⴰ'
    # pb0 = r'.'
    # pc0 = sub(pa0, pb0, c33)
    pa1 = r',|٬|٫|‚|，'
    pb1 = r'،'
    pc1 = sub(pa1, pb1, c33)
    pa2 = r'ʕ'
    pb2 = r'؟'
    pc2 = sub(pa2, pb2, pc1)
    na0 = r'۰|٠'
    nb0 = r'0'
    nc0 = sub(na0, nb0, pc2)
    na1 = r'۱|١'
    nb1 = r'1'
    nc1 = sub(na1, nb1, nc0)
    na2 = r'۲|٢'
    nb2 = r'2'
    nc2 = sub(na2, nb2, nc1)
    na3 = r'۳|٣'
    nb3 = r'3'
    nc3 = sub(na3, nb3, nc2)
    na4 = r'۴|٤'
    nb4 = r'4'
    nc4 = sub(na4, nb4, nc3)
    na5 = r'۵'
    nb5 = r'5'
    nc5 = sub(na5, nb5, nc4)
    na6 = r'۶|٦'
    nb6 = r'6'
    nc6 = sub(na6, nb6, nc5)
    na7 = r'۷|٧'
    nb7 = r'7'
    nc7 = sub(na7, nb7, nc6)
    na8 = r'۸|٨'
    nb8 = r'8'
    nc8 = sub(na8, nb8, nc7)
    na9 = r'۹|٩'
    nb9 = r'9'
    nc9 = sub(na9, nb9, nc8)
    ea1 = r'ـ|ِ|ُ|َ|ٍ|ٌ|ً|'
    eb1 = r''
    ec1 = sub(ea1, eb1, nc9)
    Sa1 = r'( )+'
    Sb1 = r' '
    Sc1 = sub(Sa1, Sb1, ec1)
    Sa2 = r'(\n)+'
    Sb2 = r'\n'
    Sc2 = sub(Sa2, Sb2, Sc1)
    Sa3 = r'ﻻ|ﻼ'
    Sb3 = r'لا'
    Sc3 = sub(Sa3, Sb3, Sc2)
    Sa4 = r'ﷲ'
    Sb4 = r'الله'
    Sc4 = sub(Sa4, Sb4, Sc3)
    return sub(zero_space, '', Sc4)

>Preparing data for training


In [ ]:
# import constants
# !pip install pyunpack
# from pyunpack import Archive

def read_csv_train_data(doc_add):
  
    csv_data = pd.read_csv(doc_add, delimiter='\t')

    doc_arr = []
    doc_id = 0

    for index in range(csv_data.shape[0]):
        text = normalizer(str(csv_data['text'][index]))+' '
        sentences = sentence_formation(text)
        for sentence in sentences:
            doc_arr.append(sentence)
        doc_id += 1
        # if doc_id % 10000 == 0:
        #     print(doc_id)
    print('####   done reading file   ####')
    return doc_arr


def report_unique_character(doc_arr):
    """
    :param doc_arr: doc's array 
    :return: chars a set of every used char in the training set.
    """
    chars = set()
    for news in doc_arr:
        for ch in news[2:-2]:
            chars.add(ch)
    print('unique chars number:\t{}'.format(len(chars)))
    return chars


def tokenizer(doc_arr):
    try:
        char2index = dict()
        char_limit = 0
        sentences_arr = list()
        # save char2index dictionary to use it to index docs
        with open(document_root_path + 'char2index.pickle', 'rb') as char2index_file:
            char2index = pickle.load(char2index_file)

        while True:
            avg_len = int(sum([len(news.split(' ')) for news in doc_arr]) / len(doc_arr))
            # extract sentences that have less or equal word  to avg_len
            for sentence in doc_arr:
                if len(sentence.split()) <= avg_len:
                    sentences_arr.append(sentence)
                    # find max char in those sentences
                    if len(sentence) > char_limit:
                        char_limit = len(sentence)
            break
        print('####   start indexing   ####')
        
        # allocate a matrix for indexed news sentences
        indexed_news = np.zeros((len(sentences_arr), char_limit), dtype=int)
        # for each sentence, put their chars index in the indexed_news
        for line, sentence in enumerate(sentences_arr):
            indexed_news[line][0] = char2index['\\n']
            for position, char in enumerate(sentence[2:-2]):
                indexed_news[line][position + 1] = char2index[char]
            indexed_news[line][len(sentence[2:-2]) + 1] = char2index['\\t']
            # if line % 10000 == 0:
            #     print(line)

        print('####   done indexing   ####')

        # write the indexed news on disk for further use
        with open(document_root_path + 'indexed_train.pickle', 'wb') as indexed_train_file:
            pickle.dump(indexed_news, indexed_train_file, protocol=pickle.HIGHEST_PROTOCOL)
        print('####   saved indexed file    #####')
    except IOError:  # there are no char2index and index2char dictionary on disk, so compute it first.
        # get texts char
        chars = report_unique_character(doc_arr)
        chars = list(chars)
        # add start and end sign to the char index dict
        chars.append('\\n')
        chars.append('\\t')
        # sort it
        chars = sorted(chars)
        # construct the char2index and index2char dict
        char2index = {ch: i for i, ch in enumerate(chars)}
        index2char = {i: ch for i, ch in enumerate(chars)}

        # save them on the disk
        with open(document_root_path + 'index2char.pickle', 'wb') as index2char_file:
            pickle.dump(index2char, index2char_file, protocol=pickle.HIGHEST_PROTOCOL)
        with open(document_root_path + 'char2index.pickle', 'wb') as char2index_file:
            pickle.dump(char2index, char2index_file, protocol=pickle.HIGHEST_PROTOCOL)
        # the requirements of this function are on the disk now, rerun the function to tokenize the news
        tokenizer(doc_arr)


def sentence_formation(news):
    sentences = list()
    end_points = sorted([r.start() for r in re.finditer('[.؟]', news)])
    start_index = 0
    for i in end_points:
        sentences.append('\\n' + news[start_index:i + 2] + '\\t')
        start_index = i + 2
    return sentences



# Archive('drive/MyDrive/Assignment_03/News.rar').extractall('drive/MyDrive/Assignment_03/data')

tokenizer(read_csv_train_data(given_doc_root_path+'train.csv'))



Define the Model and Train it

In [17]:
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# define the RNN class which will be used as the model
class RNN(nn.Module):
    def __init__(self, input_size, output_size, hidden_size, num_layers):
        super(RNN, self).__init__()
        self.embedding = nn.Embedding(input_size, input_size)
        self.rnn = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
    
    def forward(self, input_seq, hidden_state):
        embedding = self.embedding(input_seq)
        output, hidden_state = self.rnn(embedding, hidden_state)
        output = self.decoder(output)
        return output, (hidden_state[0].detach(), hidden_state[1].detach())
    
    
def train():
    ########### Hyperparameters ###########
    hidden_size = 256   # size of hidden state
    num_layers = 2      # num of layers in LSTM layer stack
    lr = 0.001          # learning rate
    epochs = 10         # max number of epochs
    load_chk = False   # load weights from save_path directory to continue training
    saved_model_path = document_root_path+'CharRNN_5.pth'
    saved_optimizer_path = document_root_path+'Evaluation_5.pth'
    #######################################
   
    # load char2index and index2char dict() using saved pickle files
    char2index = dict()
    index2char = dict()
    with open(document_root_path+'char2index.pickle','rb') as char2index_file:
      char2index = pickle.load(char2index_file,encoding='utf8')
    with open(document_root_path+'index2char.pickle', 'rb') as index2char_file:
      index2char = pickle.load(index2char_file,encoding='utf8')
    
    # a list of character used in the text
    chars = list(char2index.keys())
    # number of all used characters
    vocab_size = len(chars)

    # load indexed data which has been saved on disk (here would be google drive)
    with open(document_root_path+'indexed_train.pickle', 'rb') as indexed_data_file:
      data = pickle.load(indexed_data_file)

    n= len(data)

    # construct an instance of the RNN model
    rnn = RNN(vocab_size, vocab_size, hidden_size, num_layers).to(device)

    if load_chk:
      rnn.load_state_dict(torch.load(saved_model_path))
      print("Model loaded successfully !!")

    # loss function and optimizer
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)
    # load optimizer if load_chk is True
    if load_chk:
      optimizer.load_state_dict(torch.load(saved_optimizer_path))
      print("Optimizer loaded successfully")

    
    # # training loop
    for i_epoch in range(1, epochs+1):
      start = time.time()
        
      running_loss = 0
      # initial hiden state
      hidden_state = None
      

      for line,sentence in enumerate(data):
        # 2 array represent input string and the target string
        input_seq = torch.tensor(sentence[:-1]).to(device)
        target_seq = torch.tensor(sentence[1:]).to(device)
        # chanbe their dimention to be capable to be used as model and loss function input
        input_seq = torch.unsqueeze(input_seq,dim=1)
        target_seq = torch.unsqueeze(target_seq,dim=1)          
        
        # forward pass
        output, hidden_state = rnn(input_seq, hidden_state)

        # compute loss
        loss = loss_fn(torch.squeeze(output), torch.squeeze(target_seq))
        running_loss += loss.item()
        
        # compute gradients and take optimizer step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # see model performance while training one epoch
        if line %100000 ==0 and line!=0:
          print("Epoch: {0} \t Loss: {1:.8f}".format(line, running_loss/line))
            
        
      # print loss and save weights after every epoch
      print("Epoch: {0} \t Loss: {1:.8f}".format(i_epoch, running_loss/n))
      torch.save(rnn.state_dict(), document_root_path+'CharRNN_{}.pth'.format(i_epoch))
      torch.save(optimizer.state_dict(), document_root_path+'Optimizer_{}.pth'.format(i_epoch))
      print('time:  ',time.time()-start)




train()

Model loaded successfully !!
Optimizer loaded successfully
Epoch: 100000 	 Loss: 0.50616135
Epoch: 200000 	 Loss: 0.48187288
Epoch: 300000 	 Loss: 0.49679281
Epoch: 400000 	 Loss: 0.51081238
Epoch: 500000 	 Loss: 0.51839627
Epoch: 600000 	 Loss: 0.48678916
Epoch: 1 	 Loss: 0.48583660
time:   9960.526891231537


KeyboardInterrupt: ignored